In [2]:
import torch
import torchvision
from torch import nn
from torchvision import transforms
from torch.utils.data import Subset
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt

# Carpeta root del dataset
image_path = 'BD_perros'

transform = transforms.Compose([transforms.ToTensor(),
                                #transforms.Resize((28,28)),
                                ])

dataset = torchvision.datasets.ImageFolder(image_path, transform=transform)

In [3]:
len(dataset)

1030

In [8]:
batch_size = 64

train_dataset = Subset(dataset, torch.arange(309))
test_dataset  = Subset(dataset, torch.arange(309, len(dataset)))

train_dl = DataLoader(train_dataset, batch_size= batch_size, shuffle= True)
valid_dl = DataLoader(test_dataset , batch_size= batch_size, shuffle= False)

In [18]:
model = nn.Sequential()
# Conv -> ReLU -> MaxPooling
model.add_module('conv1', nn.Conv2d(in_channels=3,out_channels=32,kernel_size=5,padding=2))
model.add_module('relu1', nn.ReLU())
model.add_module('pool1', nn.MaxPool2d(kernel_size=2))
# Conv -> ReLU -> MaxPooling
model.add_module('conv2', nn.Conv2d(in_channels=32,out_channels=64,kernel_size=5,padding=2))
model.add_module('relu2', nn.ReLU())
model.add_module('pool2', nn.MaxPool2d(kernel_size=2))
# Flatten
model.add_module('flatten', nn.Flatten())
# Full Connected -> ReLU -> Dropout
model.add_module('fc1', nn.Linear(3136,1024))
model.add_module('relu3', nn.ReLU())
model.add_module('dropout', nn.Dropout(p= .5))
# Full Connected
model.add_module('fc2', nn.Linear(1024, 10))
#model.add_module('soft', nn.Softmax(dim=10))

In [14]:
# Selección de la unidad de procesamiento
processing_unit = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(processing_unit)
model.to(device)
print(f"You're using: {device} as device.")

You're using: cuda as device.


In [15]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr= .001)

In [16]:
def train(model, num_epochs, train_dl, valid_dl, device):
    loss_hist_train = torch.zeros(num_epochs).to(device)
    accuracy_hist_train = torch.zeros(num_epochs).to(device)
 
    loss_hist_valid = torch.zeros(num_epochs).to(device)
    accuracy_hist_valid = torch.zeros(num_epochs).to(device)
 
    for epoch in range(num_epochs):
        model.train()
        for x_batch, y_batch in train_dl:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            pred = model(x_batch)
            loss = loss_fn(pred, y_batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            loss_hist_train[epoch] += loss.item() * y_batch.size(0)
            is_correct = (torch.argmax(pred, dim=1) == y_batch).float()
            accuracy_hist_train[epoch] += is_correct.sum()

        loss_hist_train[epoch] /= len(train_dl.dataset)
        accuracy_hist_train[epoch] /= len(train_dl.dataset)

        model.eval()
        with torch.no_grad():
            for x_batch, y_batch in valid_dl:
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                pred = model(x_batch)
                loss = loss_fn(pred, y_batch)
                loss_hist_valid[epoch] += loss.item() * y_batch.size(0)
                is_correct = (torch.argmax(pred, dim=1) == y_batch).float()
                accuracy_hist_valid[epoch] += is_correct.sum()
            loss_hist_valid[epoch] /= len(valid_dl.dataset)
            accuracy_hist_valid[epoch] /= len(valid_dl.dataset)
        print(f'Epoch {epoch+1} accuracy: {accuracy_hist_train[epoch]:.4f} 'f'val_accuracy: {accuracy_hist_valid[epoch]:.4f}')
    return loss_hist_train.cpu(), loss_hist_valid.cpu(), accuracy_hist_train.cpu(), accuracy_hist_valid.cpu()

In [19]:
torch.manual_seed(1)
num_epochs = 4
hist = train(model, num_epochs, train_dl, valid_dl, device)

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same